In [ ]:
import json
import pandas as pd 
import numpy as np 
from glob import glob
import os
import matplotlib.pyplot as plt
import shutil

In [ ]:
folder_path=glob("../../data/*")
data1_img_path=glob(folder_path[0]+"/data/*.jpg")
data2_img_path=glob(folder_path[1]+"/data/*.jpg")
data3_img_path=glob(folder_path[2]+"/data/*.jpg")
data4_img_path=glob(folder_path[3]+"/data/*.jpg")

def AnnotationsPd(annotations):
    values_list = []
    names_list = [] 
    df = pd.DataFrame(annotations['tags'])
    for row in df['attributes']:
        values_list.append(row[0]['value'])
        names_list.append(row[0]['name'])

    df=df.drop(['attributes'],axis=1)   
    df.insert(3,'value',values_list)
    df.insert(4,'name',names_list)
    return df



In [ ]:
file1=[]
file2=[]
file3=[]
file4=[]
with open(folder_path[0]+'/annotations.json') as f:
    annotations_json = json.load(f)
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data1_img_path if str(frame).zfill(5) in s] 
    file1.append(file[0])
file1=list(set(file1))

with open(folder_path[1]+'/annotations.json') as f:
    annotations_json = json.load(f)   
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data2_img_path if str(frame).zfill(5) in s] 
    file2.append(file[0])
file2=list(set(file2))

with open(folder_path[2]+'/annotations.json') as f:
    annotations_json = json.load(f)    
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data3_img_path if str(frame).zfill(5) in s] 
    file3.append(file[0])
file3=list(set(file3))

with open(folder_path[3]+'/annotations.json') as f:
    annotations_json = json.load(f)    
std_frame=AnnotationsPd(annotations_json[0])['frame']
for frame in std_frame:
    file= [s for s in data4_img_path if str(frame).zfill(5) in s] 
    file4.append(file[0])
file4=list(set(file4))

In [ ]:
standard_file=[]
TF_standard=[]
for i in range(len(file1)):
    shutil.copy(file1[i],'../../data/original_dataset/StandardFrame/data1/'+str(i)+'.jpg')
    standard_file.append('StandardFrame/data1/'+str(i)+'.jpg')
    TF_standard.append(1)
for i in range(len(file2)):
    shutil.copy(file2[i],'../../data/original_dataset/StandardFrame/data2/'+str(i)+'.jpg')
    standard_file.append('StandardFrame/data1/'+str(i)+'.jpg')
    TF_standard.append(1)
for i in range(len(file3)):
    shutil.copy(file3[i],'../../data/original_dataset/StandardFrame/data3/'+str(i)+'.jpg')
    standard_file.append('StandardFrame/data1/'+str(i)+'.jpg')
    TF_standard.append(1)
for i in range(len(file4)):
    shutil.copy(file4[i],'../../data/original_dataset/StandardFrame/data4/'+str(i)+'.jpg')
    standard_file.append('StandardFrame/data1/'+str(i)+'.jpg')
    TF_standard.append(1)
    

In [70]:
not_standard_file1=list(set(data1_img_path)-set(file1))
not_standard_file2=list(set(data2_img_path)-set(file2))
not_standard_file3=list(set(data3_img_path)-set(file3))
not_standard_file4=list(set(data4_img_path)-set(file4))

for i in range(len(not_standard_file1)):
    shutil.copy(not_standard_file1[i],'../../data/original_dataset/NotStandardFrame/data1/'+str(i)+'.jpg')
    standard_file.append('NotStandardFrame/data1/'+str(i)+'.jpg')
    TF_standard.append(0)
for i in range(len(not_standard_file2)):
    shutil.copy(not_standard_file2[i],'../../data/original_dataset/NotStandardFrame/data2/'+str(i)+'.jpg')
    standard_file.append('NotStandardFrame/data2/'+str(i)+'.jpg')
    TF_standard.append(0)
for i in range(len(not_standard_file3)):
    shutil.copy(not_standard_file3[i],'../../data/original_dataset/NotStandardFrame/data3/'+str(i)+'.jpg')
    standard_file.append('NotStandardFrame/data3/'+str(i)+'.jpg')
    TF_standard.append(0)
for i in range(len(not_standard_file4)):
    shutil.copy(not_standard_file4[i],'../../data/original_dataset/NotStandardFrame/data4/'+str(i)+'.jpg')
    standard_file.append('NotStandardFrame/data4/'+str(i)+'.jpg')
    TF_standard.append(0)

In [73]:
original_standard_frame = pd.DataFrame({'file_path':standard_file,
                             'standard_frame':TF_standard})


In [74]:
original_standard_frame.to_csv('../../data/original_dataset/data_set.csv', index = None) #csv파일로 생성